There are four features used in this regression:
* one binary feature (whether or not the house has covered parking)
* one numerical feature (size, measure in thousands of square meters)
* two categorical features (architectural style and location)

In [ ]:
# import libraries
import pathlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch 
from torch import nn, optim, utils

In [ ]:
# download dataset
!gdown 1hLxbWVrnmSnZDGjvu8HeQex6ezbXeLB6

In [ ]:
# Load Data
features = pd.read_csv(
    "house_prices.txt",
    delimiter = "\t",
    skiprows=[0, 1, 2, 3, 4],
    header=None,
    usecols=[0, 1, 2, 3, 4, 6, 7, 8],
    names=["parking", "sq_meters", "art_deco", "bungalow", "colonial", "west", "east", "north"],
    dtype=np.float32
)

target = pd.read_csv(
    "house_prices.txt",
    delimiter = "\t",
    skiprows=[0, 1, 2, 3, 4],
    header=None,
    usecols=[5],
    names=["price"],
    dtype=np.float32
)

# display the first 5 rows of the dataset
features.head()



### Split the dataset into training and testing sets

In [ ]:
def split_data(features, target, train_ratio):
    # Determine the number of samples for training
    num_train_samples = int(train_ratio * features.shape[0])

    # Create an array of indices for shuffling
    index = np.arange(features.shape[0])
    np.random.shuffle(index)

    # Split indices into training and testing indeces
    train_index = index[:num_train_samples]
    test_index = index[num_train_samples:]

    # Split features and target using indices and convert to numPy-Array
    features_train = features[train_index]
    features_test = features[test_index]
    target_train = target[train_index]
    target_test = target[test_index]

    return features_train, features_test, target_train, target_test

In [ ]:
# Define train-ratio and split the data using the function split_data
train_ratio = 0.8
features_train, features_test, target_train, target_test = split_data(features.to_numpy(), target.to_numpy(), train_ratio)

# create datasets
train_features_tensor = torch.from_numpy(features_train)
train_target_tensor = torch.from_numpy(target_train)

test_features_tensor = torch.from_numpy(features_test)
test_target_tensor = torch.from_numpy(target_test)

# create datasets for training and testing
train_dataset = utils.data.TensorDataset(train_features_tensor, train_target_tensor)
test_dataset = utils.data.TensorDataset(test_features_tensor, test_target_tensor)

# create dataloaders, batch_size = number of samples
train_dataloader = utils.data.DataLoader(train_dataset, batch_size=40, shuffle=True)
test_dataloader = utils.data.DataLoader(test_dataset, batch_size=40, shuffle=False)

### Define the NN model

In [ ]:
class NeuralNetwork(nn.Module): # inherit from the bas class
    def __init__(self, input_size, hidden_size):
        super(NeuralNetwork, self).__init__() # constructor of base class
        self.hidden1 = nn.Linear(input_size, hidden_size) # create first hidden layer
        self.hidden2 = nn.Linear(hidden_size, hidden_size) # create second hidden layer
        self.relu = nn.ReLU() # activation function
        self.output_layer = nn.Linear(hidden_size, 1) # maps the hidden layers output to a single value

    # input data flow trough the network
    def forward(self, x):
        x = self.relu(self.hidden1(x)) # ReLU activation
        x = self.relu(self.hidden2(x)) # ReLU activation
        x = self.output_layer(x) # output layer
        return x

### Set training and assessment function

In [ ]:
# Define the training function
def train_model(model, train_dataloader, test_dataloader, criterion, optimizer, num_epochs, device):
    train_loss_values = []
    test_loss_values = []

    for epoch in range(num_epochs):
        model.train()  # Set the model in training mode
        total_loss = 0  # Initialize total_loss

        for features, targets in train_dataloader:
            features = features.to(device)
            targets = targets.to(device)

            optimizer.zero_grad() # resetting gradients before computing new ones
            outputs = model(features) # predictions
            loss = criterion(outputs, targets) # calculating loss between pred and actual

            loss.backward() # compute gradients of the model with respect to the loss
            optimizer.step() # adjust model parameters (weights)

            total_loss += loss.item() # apply loss to total

        avg_loss = total_loss / len(train_dataloader)
        train_loss_values.append(avg_loss)  # Log the training loss value for this epoch

        if (epoch + 1) % 50 == 0:
            print(f"Iteration {epoch+1}/{num_epochs} - Loss: {avg_loss:.4f}")

        # Calculate test loss for each epoch but with gradient calulation disabled
        test_loss = assess_performance(model, test_dataloader, criterion, device)
        test_loss_values.append(test_loss)

    return train_loss_values, test_loss_values


# Define a function for performance assessment
def assess_performance(model, dataloader, criterion, device):
    model.eval()  # Set the model to evaluation mode
    total_loss = 0
    # disable gradient calculation
    with torch.no_grad():
        for inputs, targets in dataloader:
            # move to gpu
            inputs = inputs.to(device)
            targets = targets.to(device)

            # forward pass
            outputs = model(inputs)

            # loss calculation
            loss = criterion(outputs, targets)
            total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    return avg_loss

### Train the model

In [ ]:
# Set the computing device (CPU/GPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Create the model neural network
hidden_size = 10  # number of neurons in the hidden layers
model = NeuralNetwork(input_size=features_train.shape[1], hidden_size=hidden_size)  # input size = number of features
model.to(device)  # Move the model to gpu or cpu
print(model)

# Hyperparameters
learning_rate = 0.001
num_epochs = 300

# Define the loss function and optimizer
criterion = nn.MSELoss()  # Mean Squared Error loss
optimizer = optim.Adam(model.parameters(), lr=learning_rate)  # Adam optimizer

# Call the training function
train_loss_values, test_loss_values = train_model(model, train_dataloader, test_dataloader, criterion, optimizer, num_epochs, device)

# Plot the loss curves
plt.plot(range(1, num_epochs + 1), train_loss_values, label="Training Loss")
plt.plot(range(1, num_epochs + 1), test_loss_values, label="Testing Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training and Testing Loss Curves")
plt.legend()
plt.show()

### Performance Test using MSE

In [ ]:
# Calculate test loss using the assessment function 
test_loss = assess_performance(model, test_dataloader, criterion, device)
train_loss = assess_performance(model, train_dataloader, criterion, device)

# Print the calculated loss values
print(f"Training Loss: {train_loss:.4f}")
print(f"Testing Loss: {test_loss:.4f}")